In [37]:
import pyspark
from pyspark.sql import SparkSession

In [38]:
pyspark.__version__

'3.5.5'

In [39]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

In [40]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [41]:
df_green.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge',

In [42]:
df_green.createOrReplaceTempView('green')

In [43]:
spark.sql("""
    SELECT date_trunc('month',lpep_pickup_datetime) from green
""").show()

+---------------------------------------+
|date_trunc(month, lpep_pickup_datetime)|
+---------------------------------------+
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
|                    2020-01-01 00:00:00|
+---------------------------------

In [49]:
df_result = spark.sql("""
SELECT 
    -- Revenue grouping 
    
    date_trunc("hour", lpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,
    
    SUM(total_amount) AS total_amount,
    COUNT(1) as number_records
    
FROM green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1,2
""")

In [50]:
df_result.show()

[Stage 56:=========>                                              (2 + 10) / 12]

+-------------------+------------+------------------+--------------+
|               hour|revenue_zone|      total_amount|number_records|
+-------------------+------------+------------------+--------------+
|2020-01-28 19:00:00|         134|193.61000000000007|            17|
|2020-01-22 19:00:00|          65| 657.0300000000001|            41|
|2020-01-27 08:00:00|          17|             85.56|             4|
|2020-01-02 09:00:00|          66|229.39999999999998|            12|
|2020-01-02 12:00:00|          89|310.28000000000003|            14|
|2020-01-07 12:00:00|          66|             179.5|             9|
|2020-01-03 08:00:00|         223|165.90000000000003|             9|
|2020-01-17 10:00:00|          41| 638.2699999999999|            49|
|2020-01-02 11:00:00|          26|             198.6|             7|
|2020-01-12 20:00:00|         247|36.900000000000006|             3|
|2020-01-12 15:00:00|          10|             75.92|             1|
|2020-01-24 04:00:00|         129|

In [51]:
df_green_hourly_total = df_result

In [60]:
df_green_hourly_total \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

25/03/05 18:28:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 18:28:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 18:28:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 18:28:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 18:28:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/05 18:28:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 18:28:56 WARN MemoryManager: Total allocation exceeds 95.

In [53]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [54]:
df_yellow.createOrReplaceTempView('yellow')

In [55]:
df_yellow_hourly_total = spark.sql("""
SELECT 
    -- Revenue grouping 
    
    date_trunc("hour", tpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,
    
    SUM(total_amount) AS total_amount,
    COUNT(1) as number_records
    
FROM yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1,2
""")

In [59]:
df_yellow_hourly_total \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

25/03/05 18:28:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 18:28:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 18:28:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 18:28:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 18:28:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/05 18:28:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 18:28:45 WARN MemoryManager: Total allocation exceeds 95.

In [66]:
df_y_temp = df_yellow_hourly_total \
    .withColumnRenamed('total_amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_count') 

In [67]:
df_g_temp = df_green_hourly_total \
    .withColumnRenamed('total_amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_count') 

In [68]:
df_joined_hourly = df_y_temp.join(df_g_temp, on = ['hour', 'revenue_zone'], how = 'outer')

In [69]:
df_joined_hourly.show()

[Stage 92:===========================================>            (10 + 3) / 13]

+-------------------+------------+-----------------+------------+------------------+-----------+
|               hour|revenue_zone|    yellow_amount|yellow_count|      green_amount|green_count|
+-------------------+------------+-----------------+------------+------------------+-----------+
|2020-01-01 00:00:00|           3|             25.0|           1|              NULL|       NULL|
|2020-01-01 00:00:00|          40|            89.97|           5|168.97999999999996|          8|
|2020-01-01 00:00:00|          45|732.4800000000002|          42|              NULL|       NULL|
|2020-01-01 00:00:00|          47|              8.3|           1|              13.3|          1|
|2020-01-01 00:00:00|          51|             31.0|           1|              17.8|          2|
|2020-01-01 00:00:00|          68|7825.070000000012|         396|              NULL|       NULL|
|2020-01-01 00:00:00|          73|             17.3|           1|              NULL|       NULL|
|2020-01-01 00:00:00|         

In [71]:
df_joined_hourly.write.parquet('data/report/revenue/total')

25/03/05 20:11:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 20:11:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 20:11:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 20:11:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 20:11:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/05 20:12:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 20:12:00 WARN MemoryManager: Total allocation exceeds 95.

In [72]:
df_joined = spark.read.parquet('data/report/revenue/total')

In [73]:
df_joined.show()

+-------------------+------------+------------------+------------+------------------+-----------+
|               hour|revenue_zone|     yellow_amount|yellow_count|      green_amount|green_count|
+-------------------+------------+------------------+------------+------------------+-----------+
|2020-01-01 00:00:00|           4|1004.3000000000002|          57|              NULL|       NULL|
|2020-01-01 00:00:00|          37|161.60999999999999|           7|            175.67|          6|
|2020-01-01 00:00:00|          62|             61.43|           1|             15.95|          1|
|2020-01-01 00:00:00|          74| 586.2100000000002|          47|317.09000000000015|         24|
|2020-01-01 00:00:00|          97|403.02000000000004|          20|            786.18|         25|
|2020-01-01 00:00:00|         127|43.900000000000006|           3|              NULL|       NULL|
|2020-01-01 00:00:00|         130|              NULL|        NULL|            133.35|          7|
|2020-01-01 00:00:00

In [74]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [75]:
df.write.parquet('zones')

In [76]:
df_zones = spark.read.parquet('zones')

In [77]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [81]:
df_results = df_joined.join(df_zones, df_joined.revenue_zone == df_zones.LocationID)

In [82]:
df_results.show()

+-------------------+------------+------------------+------------+------------------+-----------+----------+---------+--------------------+------------+
|               hour|revenue_zone|     yellow_amount|yellow_count|      green_amount|green_count|LocationID|  Borough|                Zone|service_zone|
+-------------------+------------+------------------+------------+------------------+-----------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|           4|1004.3000000000002|          57|              NULL|       NULL|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|          37|161.60999999999999|           7|            175.67|          6|        37| Brooklyn|      Bushwick South|   Boro Zone|
|2020-01-01 00:00:00|          62|             61.43|           1|             15.95|          1|        62| Brooklyn| Crown Heights South|   Boro Zone|
|2020-01-01 00:00:00|          74| 586.2100000000002|          47|317.090000000000

In [83]:
df_result.drop('LocationID').write.parquet('tmp/revenue-zones')

25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/05 20:31:43 WARN MemoryManager: Total allocation exceeds 95.